In [210]:
import os
import pandas as pd
import numpy as np
import numpy.char as char
from sklearn.preprocessing import OneHotEncoder

In [211]:
!pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\pjroc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
with open('../data/TOI_2025.10.03_12.20.22.csv') as toi:
    toi_df = pd.read_csv(toi)

,toi,tid,tfopwg_disp,rastr,ra,decstr,dec,st_pmra,st_pmraerr1,st_pmraerr2,...,st_logg,st_loggerr1,st_loggerr2,st_logglim,st_rad,st_raderr1,st_raderr2,st_radlim,toi_created,rowupdate
0,1000.01,50365310,FP,07h29m25.85s,112.357708,-12d41m45.46s,-12.695960,-5.964,0.085,-0.085,...,4.19,0.07,-0.07,0,2.169860,0.072573,-0.072573,0,24/07/2019 15:58,09/09/2024 10:08
1,1001.01,88863718,PC,08h10m19.31s,122.580465,-05d30m49.87s,-5.513852,-4.956,0.102,-0.102,...,4.03,0.09,-0.09,0,2.010000,0.090000,-0.090000,0,24/07/2019 15:58,03/04/2023 14:31
2,1002.01,124709665,FP,06h58m54.47s,104.726966,-10d34m49.64s,-10.580455,-1.462,0.206,-0.206,...,NaN,NaN,NaN,0,5.730000,NaN,NaN,0,24/07/2019 15:58,11/07/2022 16:02
3,1003.01,106997505,FP,07h22m14.39s,110.559945,-25d12m25.26s,-25.207017,-0.939,0.041,-0.041,...,4.15,1.64,-1.64,0,NaN,NaN,NaN,0,24/07/2019 15:58,23/02/2022 10:10
4,1004.01,238597883,FP,08h08m42.77s,122.178195,-48d48m10.12s,-48.802811,-4.496,0.069,-0.069,...,4.14,0.07,-0.07,0,2.150000,0.060000,-0.060000,0,24/07/2019 15:58,09/09/2024 10:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7698,995.01,317951248,FP,07h23m14.75s,110.811443,+05d33m46.26s,5.562850,2.061,0.405,-0.405,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,24/07/2019 15:58,29/10/2021 12:59
7699,996.01,142918609,FP,07h57m23.99s,119.349948,-19d30m57.65s,-19.516015,-3.900,0.848,-0.848,...,NaN,NaN,NaN,0,2.050000,NaN,NaN,0,24/07/2019 15:58,29/10/2021 12:59
7700,997.01,341729521,FP,08h05m16.69s,121.319521,-59d34m47.27s,-59.579798,-44.770,0.044,-0.044,...,4.52,0.08,-0.08,0,0.926261,0.045789,-0.045789,0,24/07/2019 15:58,09/09/2024 10:08
7701,998.01,54390047,FP,07h53m16.69s,118.319555,-14d13m07.76s,-14.218823,-1.706,0.069,-0.069,...,4.01,0.07,-0.07,0,2.349860,0.091578,-0.091578,0,24/07/2019 15:58,09/09/2024 10:08


In [ ]:
###this drops columns with >90% null values
toi_df = toi_df.dropna(thresh=len(toi_df)*0.1, axis=1)

,toi,tid,tfopwg_disp,rastr,ra,decstr,dec,st_pmra,st_pmraerr1,st_pmraerr2,...,st_logg,st_loggerr1,st_loggerr2,st_logglim,st_rad,st_raderr1,st_raderr2,st_radlim,toi_created,rowupdate
0,1000.01,50365310,FP,07h29m25.85s,112.357708,-12d41m45.46s,-12.695960,-5.964,0.085,-0.085,...,4.19,0.07,-0.07,0,2.169860,0.072573,-0.072573,0,24/07/2019 15:58,09/09/2024 10:08
1,1001.01,88863718,PC,08h10m19.31s,122.580465,-05d30m49.87s,-5.513852,-4.956,0.102,-0.102,...,4.03,0.09,-0.09,0,2.010000,0.090000,-0.090000,0,24/07/2019 15:58,03/04/2023 14:31
2,1002.01,124709665,FP,06h58m54.47s,104.726966,-10d34m49.64s,-10.580455,-1.462,0.206,-0.206,...,NaN,NaN,NaN,0,5.730000,NaN,NaN,0,24/07/2019 15:58,11/07/2022 16:02
3,1003.01,106997505,FP,07h22m14.39s,110.559945,-25d12m25.26s,-25.207017,-0.939,0.041,-0.041,...,4.15,1.64,-1.64,0,NaN,NaN,NaN,0,24/07/2019 15:58,23/02/2022 10:10
4,1004.01,238597883,FP,08h08m42.77s,122.178195,-48d48m10.12s,-48.802811,-4.496,0.069,-0.069,...,4.14,0.07,-0.07,0,2.150000,0.060000,-0.060000,0,24/07/2019 15:58,09/09/2024 10:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7698,995.01,317951248,FP,07h23m14.75s,110.811443,+05d33m46.26s,5.562850,2.061,0.405,-0.405,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,24/07/2019 15:58,29/10/2021 12:59
7699,996.01,142918609,FP,07h57m23.99s,119.349948,-19d30m57.65s,-19.516015,-3.900,0.848,-0.848,...,NaN,NaN,NaN,0,2.050000,NaN,NaN,0,24/07/2019 15:58,29/10/2021 12:59
7700,997.01,341729521,FP,08h05m16.69s,121.319521,-59d34m47.27s,-59.579798,-44.770,0.044,-0.044,...,4.52,0.08,-0.08,0,0.926261,0.045789,-0.045789,0,24/07/2019 15:58,09/09/2024 10:08
7701,998.01,54390047,FP,07h53m16.69s,118.319555,-14d13m07.76s,-14.218823,-1.706,0.069,-0.069,...,4.01,0.07,-0.07,0,2.349860,0.091578,-0.091578,0,24/07/2019 15:58,09/09/2024 10:08


In [ ]:
toi_df = toi_df.drop(columns=["toi", "tid", "toi_created", "rowupdate"])

,tfopwg_disp,rastr,ra,decstr,dec,st_pmra,st_pmraerr1,st_pmraerr2,st_pmralim,st_pmdec,...,st_tefferr2,st_tefflim,st_logg,st_loggerr1,st_loggerr2,st_logglim,st_rad,st_raderr1,st_raderr2,st_radlim
0,FP,07h29m25.85s,112.357708,-12d41m45.46s,-12.695960,-5.964,0.085,-0.085,0.0,-0.076,...,-264.7,0,4.19,0.07,-0.07,0,2.169860,0.072573,-0.072573,0
1,PC,08h10m19.31s,122.580465,-05d30m49.87s,-5.513852,-4.956,0.102,-0.102,0.0,-15.555,...,-126.4,0,4.03,0.09,-0.09,0,2.010000,0.090000,-0.090000,0
2,FP,06h58m54.47s,104.726966,-10d34m49.64s,-10.580455,-1.462,0.206,-0.206,0.0,-2.249,...,-124.0,0,NaN,NaN,NaN,0,5.730000,NaN,NaN,0
3,FP,07h22m14.39s,110.559945,-25d12m25.26s,-25.207017,-0.939,0.041,-0.041,0.0,1.640,...,-567.0,0,4.15,1.64,-1.64,0,NaN,NaN,NaN,0
4,FP,08h08m42.77s,122.178195,-48d48m10.12s,-48.802811,-4.496,0.069,-0.069,0.0,9.347,...,-171.1,0,4.14,0.07,-0.07,0,2.150000,0.060000,-0.060000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7698,FP,07h23m14.75s,110.811443,+05d33m46.26s,5.562850,2.061,0.405,-0.405,0.0,-7.082,...,-46.5,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0
7699,FP,07h57m23.99s,119.349948,-19d30m57.65s,-19.516015,-3.900,0.848,-0.848,0.0,3.866,...,NaN,0,NaN,NaN,NaN,0,2.050000,NaN,NaN,0
7700,FP,08h05m16.69s,121.319521,-59d34m47.27s,-59.579798,-44.770,0.044,-0.044,0.0,-2.616,...,-132.9,0,4.52,0.08,-0.08,0,0.926261,0.045789,-0.045789,0
7701,FP,07h53m16.69s,118.319555,-14d13m07.76s,-14.218823,-1.706,0.069,-0.069,0.0,-1.714,...,-130.7,0,4.01,0.07,-0.07,0,2.349860,0.091578,-0.091578,0


In [215]:
# Function to convert Right Ascension string to seconds
def time_to_seconds(time_str):
    """Convert Right Ascension string (HHhMMmSS.sss) to total seconds"""
    if pd.isna(time_str):
        return None
    try:
        # Extract hours, minutes, seconds using regex
        pattern = r'(\d+)h(\d+)m([\d.]+)s'
        match = re.match(pattern, str(time_str))
        if not match:
            print(f"Warning: Could not parse RA value: {time_str}")
            return None
        hours = float(match.group(1))
        minutes = float(match.group(2))
        seconds = float(match.group(3))
        
        # Validate ranges for RA (0-24h)
        if not (0 <= hours < 24 and 0 <= minutes < 60 and 0 <= seconds < 60):
            print(f"Warning: RA values out of range: {time_str}")
            return None
            
        # Convert everything to seconds
        total_seconds = (hours * 3600) + (minutes * 60) + seconds
        return total_seconds
    except (ValueError, AttributeError) as e:
        print(f"Error converting {time_str}: {str(e)}")
        return None

toi_df['rastr'] = toi_df['rastr'].apply(time_to_seconds)


In [ ]:
# Function to convert Declination string to decimal degrees
def dec_to_degrees(dec_str):
    """Convert Declination string (DDdMMmSS.sss) to decimal degrees"""
    if pd.isna(dec_str):
        return None
    try:
        # Extract degrees, minutes, seconds using regex
        pattern = r'([+-]?\d+)d(\d+)m([\d.]+)s'
        match = re.match(pattern, str(dec_str))
        if not match:
            print(f"Warning: Could not parse DEC value: {dec_str}")
            return None
        
        degrees = float(match.group(1))
        minutes = float(match.group(2))
        seconds = float(match.group(3))
        
        # Validate ranges for DEC (-90 to +90 degrees)
        if not (-90 <= degrees <= 90 and 0 <= minutes < 60 and 0 <= seconds < 60):
            print(f"Warning: DEC values out of range: {dec_str}")
            return None
        
        # Convert to decimal degrees
        dec_degrees = degrees + (minutes/60) + (seconds/3600)
        if dec_degrees < -90 or dec_degrees > 90:
            print(f"Warning: Final DEC value out of range: {dec_degrees}")
            return None
            
        return dec_degrees
        
    except (ValueError, AttributeError) as e:
        print(f"Error converting {dec_str}: {str(e)}")
        return None

# Convert decstr to decimal degrees
print("Converting DEC strings to decimal degrees...")
print("\nSample of original values:")
print(toi_df['decstr'].head())

toi_df['decstr'] = toi_df['decstr'].apply(dec_to_degrees)


Converting DEC strings to decimal degrees...

Sample of original values:
0    -12d41m45.46s
1    -05d30m49.87s
2    -10d34m49.64s
3    -25d12m25.26s
4    -48d48m10.12s
Name: decstr, dtype: object

Sample of converted values (in decimal degrees):
0   -11.304039
1    -4.486147
2    -9.419544
3   -24.792983
4   -47.197189
Name: decstr, dtype: float64

Successfully converted: 7703 out of 7703 values (100.0%)
Range: -88.5285° to 89.0869°


In [226]:
# Show unique values in tfopwg_disp column before encoding
print("Unique values in tfopwg_disp before encoding:")
print(toi_df['tfopwg_disp'].value_counts(dropna=False))

# One-hot encode just tfopwg_disp column
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Reshape the column for OneHotEncoder (it expects 2D array)
# Fill NaN with 'missing' to explicitly encode missing values
tfopwg_data = toi_df['tfopwg_disp'].fillna('missing').values.reshape(-1, 1)

# Fit and transform the data
encoded_data = encoder.fit_transform(tfopwg_data)

# Create new column names with prefix
feature_names = [f'tfopwg_disp_{cat}' for cat in encoder.categories_[0]]

# Create DataFrame with encoded columns
encoded_df = pd.DataFrame(
    encoded_data,
    columns=feature_names,
    index=toi_df.index
)

# Add the encoded columns back to original DataFrame
result_df = pd.concat([toi_df, encoded_df], axis=1)

# Show the results

result_df

Unique values in tfopwg_disp before encoding:
tfopwg_disp
PC     4679
FP     1197
CP      684
KP      583
APC     462
FA       98
Name: count, dtype: int64


,tfopwg_disp,rastr,ra,decstr,dec,st_pmra,st_pmraerr1,st_pmraerr2,st_pmralim,st_pmdec,...,st_rad,st_raderr1,st_raderr2,st_radlim,tfopwg_disp_APC,tfopwg_disp_CP,tfopwg_disp_FA,tfopwg_disp_FP,tfopwg_disp_KP,tfopwg_disp_PC
0,FP,26965.85,112.357708,-11.304039,-12.695960,-5.964,0.085,-0.085,0.0,-0.076,...,2.169860,0.072573,-0.072573,0,0.0,0.0,0.0,1.0,0.0,0.0
1,PC,29419.31,122.580465,-4.486147,-5.513852,-4.956,0.102,-0.102,0.0,-15.555,...,2.010000,0.090000,-0.090000,0,0.0,0.0,0.0,0.0,0.0,1.0
2,FP,25134.47,104.726966,-9.419544,-10.580455,-1.462,0.206,-0.206,0.0,-2.249,...,5.730000,NaN,NaN,0,0.0,0.0,0.0,1.0,0.0,0.0
3,FP,26534.39,110.559945,-24.792983,-25.207017,-0.939,0.041,-0.041,0.0,1.640,...,NaN,NaN,NaN,0,0.0,0.0,0.0,1.0,0.0,0.0
4,FP,29322.77,122.178195,-47.197189,-48.802811,-4.496,0.069,-0.069,0.0,9.347,...,2.150000,0.060000,-0.060000,0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7698,FP,26594.75,110.811443,5.562850,5.562850,2.061,0.405,-0.405,0.0,-7.082,...,NaN,NaN,NaN,0,0.0,0.0,0.0,1.0,0.0,0.0
7699,FP,28643.99,119.349948,-18.483986,-19.516015,-3.900,0.848,-0.848,0.0,3.866,...,2.050000,NaN,NaN,0,0.0,0.0,0.0,1.0,0.0,0.0
7700,FP,29116.69,121.319521,-58.420203,-59.579798,-44.770,0.044,-0.044,0.0,-2.616,...,0.926261,0.045789,-0.045789,0,0.0,0.0,0.0,1.0,0.0,0.0
7701,FP,28396.69,118.319555,-13.781178,-14.218823,-1.706,0.069,-0.069,0.0,-1.714,...,2.349860,0.091578,-0.091578,0,0.0,0.0,0.0,1.0,0.0,0.0


In [221]:
# Separate features (X) and target variables (y)
# Get unique encoded disposition columns (removing duplicates)
target_columns = list(set(['tfopwg_disp_APC', 'tfopwg_disp_CP', 'tfopwg_disp_FA', 
                          'tfopwg_disp_FP', 'tfopwg_disp_PC']))

print("Target columns:", target_columns)

# Create target variable DataFrame
y = result_df[target_columns]

# Create feature DataFrame (excluding target and original category)
x = result_df.drop(columns=target_columns + ['tfopwg_disp'])

print("\nShape of X (features):", x.shape)
print("Shape of y (targets):", y.shape)

Target columns: ['tfopwg_disp_PC', 'tfopwg_disp_CP', 'tfopwg_disp_FP', 'tfopwg_disp_APC', 'tfopwg_disp_FA']

Shape of X (features): (7703, 55)
Shape of y (targets): (7703, 5)


In [222]:
corr_matrix = x.corr()
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column].abs() > 0.7)]
x = x.drop(columns=to_drop)


In [223]:
# Validate final dataset
print("=== Dataset Validation ===")
print("\nFeature set (X):")
print("Shape:", x.shape)
print("\nSelected features:")
for col in x.columns:
    non_null = x[col].count()
    dtype = x[col].dtype
    print(f"- {col}: {dtype} (non-null: {non_null})")

print("\nTarget set (y):")
print("Shape:", y.shape)
print("\nTarget distributions:")
for col in y.columns:
    print(f"\n{col}:")
    print(y[col].value_counts(normalize=True).mul(100).round(2))

=== Dataset Validation ===

Feature set (X):
Shape: (7703, 38)

Selected features:
- rastr: float64 (non-null: 7703)
- decstr: float64 (non-null: 7703)
- st_pmra: float64 (non-null: 7569)
- st_pmraerr1: float64 (non-null: 7569)
- st_pmralim: float64 (non-null: 7569)
- st_pmdec: float64 (non-null: 7569)
- st_pmdeclim: float64 (non-null: 7569)
- pl_tranmid: float64 (non-null: 7703)
- pl_tranmiderr1: float64 (non-null: 7692)
- pl_tranmidlim: int64 (non-null: 7703)
- pl_orbper: float64 (non-null: 7596)
- pl_orbpererr1: float64 (non-null: 7572)
- pl_orbperlim: int64 (non-null: 7703)
- pl_trandurh: float64 (non-null: 7703)
- pl_trandurherr1: float64 (non-null: 7690)
- pl_trandurhlim: int64 (non-null: 7703)
- pl_trandep: float64 (non-null: 7703)
- pl_trandeperr1: float64 (non-null: 7697)
- pl_trandeplim: int64 (non-null: 7703)
- pl_rade: float64 (non-null: 7197)
- pl_radeerr1: float64 (non-null: 6080)
- pl_radelim: int64 (non-null: 7703)
- pl_insol: float64 (non-null: 7527)
- pl_eqt: float64 